In [4]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

router_llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

model = HuggingFaceEndpoint(repo_id="deepseek-ai/DeepSeek-V3.2")
main_llm = ChatHuggingFace(llm=model)

In [11]:
from langchain_core.prompts import PromptTemplate,load_prompt
from pydantic import Field,BaseModel
from typing import Literal

router_template = """
    You are an expert router for a Fraud Detection RAG system.
    Analyze the user's input.
"""

template=PromptTemplate(
    template=router_template,
    input_variables=['query']
)

template.save('./router_prompt.json')

class output(BaseModel):
    Decision:Literal['SEARCH','CHAT']=Field(description="Choose 'SEARCH' if the query requires technical fraud detection info. Choose 'CHAT' for general conversation.")
    query:str=Field(default="N/A",description="If decision is SEARCH, extract the core technical keywords. If CHAT, return 'N/A'.")

structured_router_llm = router_llm.with_structured_output(output)

template = load_prompt('./router_prompt.json')

router_chain = template | structured_router_llm

query = "hi what is up!"
result = dict(router_chain.invoke({'query':query}))

In [ ]:
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

chat_history=[
    SystemMessage(content='you are a chatbot name xoro')
]

while(True):
    query = input(str('enter your query'))
    chat_history.append(HumanMessage(content=query))
    result = llm.invoke(chat_history)
    print(result.content)
    chat_history.append(AIMessage(content=result.content))